# Create OMMX Adapters

OMMX is designed to exchange instance between modelers and solvers. This means each modeler needs to export their model into an [ommx.v1.Instance] and each solver needs to load an [ommx.v1.Instance] to solve it. We hope existing modelers and solvers implement their own exporter and loader of [ommx.v1.Instance], but it is possible to implement external libraries to translate the modeler-specific model between [ommx.v1.Instance]. Such libraries are called OMMX "Adapter".

[ommx-python-mip-adapeter](https://pypi.org/project/ommx-python-mip-adapter/) is an example of OMMX Adapter for [Python-MIP](https://www.python-mip.com/). This library translates the [Python-MIP's Model](https://python-mip.readthedocs.io/en/latest/classes.html#model) to [ommx.v1.Instance] and vice versa.

This notebook shows how to create an OMMX Adapter for a modeler-specific model.

[ommx.v1.Instance]: https://jij-inc.github.io/ommx/python/ommx/autoapi/ommx/v1/index.html#ommx.v1.Instance

In [1]:
import ommx.v1
from ommx.artifact import Artifact

## Raw interface
OMMX Python SDK provides a two level interface, raw interface and high-level interface.

- The high-level interface provides a user-friendly interface for users who using OMMX-supported modelers and solvers.
- The raw interface provides a low-level interface for users who want to implement their own OMMX Adapter. This exposes the interface to protocol buffer messages directly.

This notebook uses the raw interface to create an OMMX Adapter. First, load an example instance of LP problem:

$$
\begin{align*}
\text{minimize} \quad & c x \\
\text{subject to} \quad & A x = b
\end{align*}
$$

with randomly generated $c \in \mathbb{R}^5$, $b \in \mathbb{R}^7$ vectors, and a matrix $A: \mathbb{R}^5 \to \mathbb{R}^7$.

In [2]:
# Load the instance from the archive
artifact = Artifact.load_archive("../data/random_lp_instance.ommx")

# High-level API
instance = artifact.instance
assert isinstance(instance, ommx.v1.Instance)

# Raw API
raw = instance.raw
assert isinstance(raw, ommx.v1.instance_pb2.Instance)

`raw` is of type `ommx.v1.instance_pb2.Instance` which is automatically generated from the protobuf definition files `*.proto`. This contains the `objective` function to be minimized, `constraints` to be satisfied, and `variables` to be optimized.

In [3]:
assert isinstance(raw.objective, ommx.v1.function_pb2.Function)

# Since OMMX can store linear, quadratic, and polynomial functions, we need to check the type of the objective
# If the function is linear, it will have a linear field
linear = raw.objective.linear
assert isinstance(linear, ommx.v1.linear_pb2.Linear)

# If the function is quadratic, it will have a quadratic field
try:
    quadratic = raw.objective.quadratic
except AttributeError:
    # If the function is not quadratic, the quadratic field will not exist
    print("The objective is not quadratic")

The linear function is represented as a list of terms. A term, e.g. $3 x_4$, is represented as a coefficient `3.0` and the index decision variable `4`. It is stored in `Linear.terms`:

In [4]:
for term in linear.terms:
    print(f"{term.id=}, {term.coefficient=}")

term.id=0, term.coefficient=-0.9785827930112365
term.id=1, term.coefficient=-0.03808169072638057
term.id=2, term.coefficient=-0.5649347710821964
term.id=3, term.coefficient=0.29050659849927296
term.id=4, term.coefficient=-0.25876722492556814


Constraint of the problem $Ax = b$ is stored in `constraints` field. Be sure that the constraint is normalized to $f(x) = 0$  or $f(x) \leq 0$ form. For the linear constraint $Ax = b$, it is represented as $f(x) = Ax - b = 0$.

In [5]:
# There are 7 constraints
assert len(raw.constraints) == 7

# Take first constraint as an example
constraint = raw.constraints[0]
assert isinstance(constraint, ommx.v1.constraint_pb2.Constraint)

# Constraint is an equality constraint
assert constraint.equality == ommx.v1.constraint_pb2.Equality.EQUALITY_EQUAL_TO_ZERO

# f(x) = Ax - b
f = constraint.function
assert isinstance(f, ommx.v1.function_pb2.Function)